# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

### Neurons only

In [ ]:
import os
import nrrd

all_cells = False

if all_cells:
    db_schema = 'atlas_ccfv3a_averaged'
    density_filename = '/gpfs/bbp.cscs.ch/project/proj84/piluso/share/Cyrille/CCFv3a/neuron_density_averaged_all_cells.nrrd'
else:
    db_schema = 'atlas_ccfv3a_averaged'
    density_filename = '/gpfs/bbp.cscs.ch/project/proj84/piluso/share/Cyrille/CCFv3a/neuron_density_averaged.nrrd'

density_data, density_header = nrrd.read(density_filename)

region_filename = os.path.join('/gpfs/bbp.cscs.ch/home/piluso/cell_atlas/05_final_run/blue_brain_atlas_pipeline/leaves_only', 'annotation_ccfv2_l23split_barrelsplit.nrrd')
region_data, region_header = nrrd.read(region_filename)

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
import math
import random
from tqdm import tqdm

try:
    space_directions = density_header['space directions']
    scale = [
        space_directions[0][0],
        space_directions[1][1],
        space_directions[2][2]
    ]
except:
    scale = [25.0, 25.0, 25.0]

voxel_volume_um = scale[0] * scale[1] * scale[2]
millimeter_cube_um = 1000 * 1000 * 1000
D = millimeter_cube_um / voxel_volume_um

print('Deleting existing cells')
with Session(engine) as session:
   session.execute('DELETE FROM %s.cell' % db_schema)
   session.commit()

guid = 0
total = 0

with Session(engine) as session:
    volume_size = density_data.shape
    for x in tqdm(range(volume_size[0])):
        for y in range(volume_size[1]):
            for z in range(volume_size[2]):
                nb_cells = math.ceil(density_data[x][y][z] / D)
                total += nb_cells
                if nb_cells > 0:
                    region_id = region_data[x][y][z]
                    for i in range(nb_cells):
                        pos_x = (x + random.random()) * scale[0]
                        pos_y = (y + random.random()) * scale[1]
                        pos_z = (z + random.random()) * scale[2]

                        session.execute(
                            'INSERT INTO %s.cell VALUES (:guid, 0, :region_id, 0, :x, :y, :z, 0.0, 0.0, 0.0, 1.0)' % db_schema,
                            {
                                'guid': int(guid),
                                'region_id': int(region_id),
                                'x': float(pos_x),
                                'y': float(pos_y),
                                'z': float(pos_z)
                            }
                        )
                        guid += 1
            session.commit()
    print(total)